# BTC Price Analysis
This notebook demonstrates how to visualize the Bitcoin price, the Fear & Greed index, and the predicted price from our simple linear regression model.

In [ ]:
import os
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


## Check for duplicates in Parquet files

In [12]:
parquet_path = '../data/processed/fear_and_greed_history_5min.parquet'
if not os.path.exists(parquet_path):
    print(f"File not found: {parquet_path}")
else:
    df_parquet = pl.read_parquet(parquet_path)
    duplicates = df_parquet.group_by('interval_end_time').count().filter(pl.col('count') > 1)
    if duplicates.height > 0:
        print(f"Found {duplicates.height} duplicate interval_end_time values in Parquet data.")
        print(duplicates)
    else:
        print("No duplicates found in Parquet data.")



No duplicates found in Parquet data.


/var/folders/n5/chnclgwx06ng9skf1f5bc5gr0000gp/T/ipykernel_64160/1460134911.py:6: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  duplicates = df_parquet.group_by('interval_end_time').count().filter(pl.col('count') > 1)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the Parquet data
parquet_path = '../data/processed/fear_and_greed_history_5min.parquet'
df = pl.read_parquet(parquet_path)

# Convert interval_end_time to numpy datetime64 for plotting
interval_times = df.select('interval_end_time').to_numpy().flatten()
interval_times = np.array(interval_times, dtype='datetime64[ms]')

btc_price = df.select('close_bitcoin_price_usd').to_numpy().flatten()
fear_greed = df.select('avg_actual_value').to_numpy().flatten()

fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Interval End Time')
ax1.set_ylabel('BTC Price', color=color)
ax1.plot(interval_times, btc_price, label='BTC Price', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Fear & Greed Index', color=color)
ax2.plot(interval_times, fear_greed, label='Fear & Greed Index', color=color, alpha=0.5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

fig.autofmt_xdate()
plt.title('BTC Price and Fear & Greed Index Over Time')
plt.tight_layout()
plt.show()
